## **Example notebook for integrated LCA package**

##### 📍 *This notebook is made to explain how to use most the functions in the  <font color="orange">IntLCA package*</font>

Written by Margarita A. Charalambous

##### **1. Importing packages**

In this version of the Integrated LCA repository, the package IntLCA should be imported from the foler above, which can be done as shown in the cell below. 

In [ ]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from IntLCA import intLCA
from IntLCA.utils.fetcher import ActivityFilter

##### **2. Setup the project**

- To setup to use the package you will need to define the name of the <font color="orange">**project**</font> you have. This name will be used to fetch characterizaton factors from Brightway. It should be the same project that you used in the Setup notebooks. 
- You also need to specify the names of the <font color="orange">**scenarios**</font> or SSP2-RCP pathways. This should be the names of the folders created in the "remind" folder after you installed the prospective databases as matrices. 
- The <font color="orange">**years**</font> are needed in order to navigate through the installation folders. 
- The <font color="orange">**base_path**</font> is the directory path to the folder "remind" where the installed CSV files are stored. The directory should end at the "export" folder created from *premise* because our packge should handle the rest. Ideally the folder created and called "remind" should not be modified.  
- The <font color="orange">**directory**</font> is needed to store the final results.
- The <font color="orange">**methods**</font> are the LCIA categories that can be used to perform calculations. 

📍 *The  <font color="orange">**project, scenarios, years and base_path**</font> are required in order to access the data that we will be based on.*

In [ ]:
project = "Integrated LCA project"
scenarios = ["SSP2-Base",]
years = [2050]
base_path = "C:/Users/mcharalam/dev/prospectiveLCA/Premise/pLCA/Enhanced-Technosphere-wide-LCA-paper/Integrated_LCA/Notebooks/Setting up/export" #--> Change this to the path where the data is stored
directory = "Results" #--> Directory to the results folder
methods = ["IPCC 2021"] #--> Name of the method to be used. 

Create the <font color="orange">**IntLCA object**</font> with the name int_lca. This objects has access to all the modules created for this work, and contain code to access, filter and modify the technosphere, as well as performing simple calculations and first level breakdown calculations, while also, saving the results. 

In [ ]:
int_lca = intLCA.IntLCA(project, scenarios, years, base_path, directory, methods)

##### **3. Access the matrices**

Using the inforamtiion setted up above we access the paths where the matrices are stored

In [ ]:
paths = int_lca.access_data()

##### **4. Create sparce matrices and index dictionaries to filter the data**

In [ ]:
A, B, A_inds, B_inds, A_inds_rev, B_inds_rev = int_lca.create_matrices(paths)

##### **5. Filtering**

To perform the modification on the sparse matrices, we first need to know which activities we want to modify, which ones we want to replace, and which ones we want to add. Moreover, we need to specfiy for which activities we want to run LCIA. 
To facilitate the filtering, we created the functionality for the user to specify the names of the activities of interest by using keywords. 

So far, we inlcude the following keywords: 
- "includes", 
- "includes_all",
- "excludes", 
- "starts",
- "reference_product", 
- "location", 
- "excluding_locations"

Below we show how to filter using the above keywords to find the activity "transport, freight, lorry, diesel, 40t gross weight, 2050, EURO-VI, long haul". Similarly you can use the other keywords.

In [ ]:
keys_to_run = [
        {"includes_all": ["transport, freight, lorry", "40t", "diesel", "long haul"], "location": "EUR"}
]

To modify the technosphere we choose to modify the market of diesel in Europe which is the fuel that is powering the truck of choice. We want to include in the market of diesel the synthetically produced diesel we created. For that we need to find the the synthetic fraction of the market and the correct level in which we can modify. The exchanges of the european market of diesel are already represented by the fueling activities, meaning the fuel is already transported to the fueling stations before it is bukered in the trucks. Since we don't want to lose this information we want to go one level inside the fueling activities to find the diesel production activity. 

In [ ]:
keys_to_modify = [
        {"includes_all": ["diesel production, synthetic, from Fischer Tropsch process", "energy allocation"], "location": "EUR"}
]

In [ ]:
keys_to_replace = [
    {"includes_all": ["Diesel, synthetic", "energy allocation"], "location": "EUR"}
]

In [ ]:
keys_to_add = [
    {"starts": "diesel production,--", "includes_all": ["PEM", "DAC", ">3MW"], "location": "EUR"},
]

Below by using the filtering function the user can export lists of activities. The activities are represented by their unique number in the technosphere and can be used lated to modify the matrix. The numbers are the identifiers of the cells in the matrix that we want to modify.

In [ ]:
exc_to_remove, act_to_modify, act_to_add, act_to_run = int_lca.filtering(A_inds, keys_to_add, keys_to_modify, keys_to_replace, keys_to_run)

##### **6. Modifying the technosphere**

In order to make changes to the technosphere matrix A we first create a copy of it so that we safely make changes without losing the original matrix.

In [ ]:
A_ = int_lca.copy_technosphere(A)

First we remove the activities that we don't want. In our case we have one activity to remove and several to modify. 

The code goes to the specific cells i.e., A_[200,1856] = 0. 

In this case, the exchange that needs to be removed. the diesel production is the 200 and the 1856 is the diesel activity at the fueling station. 

📍 *Its important to note that all the diesel production activities filtered are automatically removed from the fueling activities*

In [ ]:
A_ = int_lca.modify_technosphere_remove(exc_to_remove, act_to_modify)

##### **7. Performing calculations**

- We can perform a simple calculation of one activity at a time, therefore we need to create a for-loop outside of the function calculate. 

In [ ]:
#First we need to add the new activity
A_ = int_lca.modify_technosphere_add(exc_to_remove[0], act_to_add, act_to_modify)


In [ ]:
FU =1
results = []
for idx in act_to_run:
    df1, contribution, s = int_lca.calculate(idx, FU)

- We can perform a breakdown calculation of one activity at a time, therefore we need to create a for-loop outside of the function calculate. 

In [ ]:
FU =1
results = []
act_tech = [i for i in A_inds_rev[act_to_add[0]]]
for idx in act_to_run:
    dfs = int_lca.breakdown(idx, FU, results, act_tech)

- Perform calculations of multiple activities with multiple methods

In [ ]:
FU =1
df = int_lca.multi_calc(act_to_run, FU)

- Perform calculations of multiple activities by modifying the technosphere everytime, while also using several methods. 

*Here we just modify the synthetic part of diesel everywhere, so that it is substituted by our technology.*

In [ ]:
FU =1
df = int_lca.overall_calc(int_lca, FU)

##### **8. Saving results**

In [ ]:
int_lca.save_results("Results", df)